# Set up

In [ ]:
!pip install albumentations
!pip install opencv-python
!pip install ultralytics
!pip install roboflow
!pip install inference supervision

In [5]:
import os, shutil
from os import listdir
from os.path import isfile, join
import cv2
import albumentations as A
import copy
import random
import numpy as np
from ultralytics import YOLO
import torch
import shutil

In [ ]:
# If you don't have yolov9 set up
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q

In [ ]:
# If you already have yolov9 in your repo
%cd yolov9

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="HV9jB1dcJZHZd1YO5S7C") 
project = rf.workspace("comp400").project("comp400-fish-detection")
dataset = project.version(3).download("yolov9")

# Train (training with on roboflow's website is easier - ignore this section)

In [ ]:
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

In [ ]:
!python3 train.py \
     --data COMP400-Fish-detection-3/data.yaml \
     --weights weights/gelan-c.pt \
     --cfg models/detect/gelan-c.yaml \
     --hyp hyp.scratch-high.yaml \
     --device cpu
     

# Deploy

In [6]:
from roboflow import Roboflow

rf = Roboflow(api_key='HV9jB1dcJZHZd1YO5S7C')
project = rf.workspace('comp400').project('comp400-fish-detection')
model = project.version(4).model

loading Roboflow workspace...
loading Roboflow project...


In [7]:
import os 

data_dir = "../data4"
file_name = "geo.txt"

with open(os.path.join(data_dir, file_name), "r") as f:
     lines = f.readlines()
     lines = lines[1:]  # Skip the header line

filenames_raw, filenames_depth = [], []

for line in lines:
     filename_raw, filename_depth, _, _, _, _, _, _ = line.strip().split()
     filenames_raw.append(filename_raw)
     filenames_depth.append(filename_depth)


In [ ]:
!mkdir predictions

In [ ]:
file = filenames_raw[0]
prediction = model.predict(os.path.join(data_dir, file), confidence=50, overlap=50).json()["predictions"][0]
x = prediction["x"]
print(prediction)

In [12]:
with open("prediction.txt", "w") as f:
     f.write("filename_raw filename_depth x y width height")
     for i in range(len(filenames_raw)):
          prediction = model.predict(os.path.join(data_dir, filenames_raw[i]), confidence=50, overlap=50).json()["predictions"]
          print(prediction)
          for j in range(len(prediction)):
               instance = prediction[j]
               x = instance["x"]
               y = instance["y"]
               width = instance["width"]
               height = instance["height"]
               f.write(f"\n{filenames_raw[i]} {filenames_depth[i]} {x} {y} {width} {height}")
     
     # uncoment next line if you want to visualize predictions
     # 

[{'x': 57.5, 'y': 31.5, 'width': 111.0, 'height': 63.0, 'confidence': 0.9175102114677429, 'class': 'fish1', 'class_id': 0, 'detection_id': 'c8597225-bdd8-4efb-9181-547de8dab27e', 'image_path': '../data4/22:28:23.171_raw.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 102.5, 'y': 66.5, 'width': 51.0, 'height': 35.0, 'confidence': 0.8794440031051636, 'class': 'fish1', 'class_id': 0, 'detection_id': 'df0bc43b-35b3-43bc-860d-791ecae7f3ad', 'image_path': '../data4/22:28:23.171_raw.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 134.0, 'y': 47.5, 'width': 46.0, 'height': 33.0, 'confidence': 0.8679935932159424, 'class': 'fish1', 'class_id': 0, 'detection_id': '8abd43bc-5ddc-4bcb-9b11-75b43008d13b', 'image_path': '../data4/22:28:23.171_raw.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 338.5, 'y': 229.5, 'width': 101.0, 'height': 35.0, 'confidence': 0.7030525207519531, 'class': 'fish1', 'class_id': 0, 'detection_id': 'f9b0b2e5-d17f-4791-a600-f43c4e769ad5', 'image_path'

In [ ]:
predict = model.predict(os.path.join(data_dir, "22:28:23.171_raw.jpg"), confidence=50, overlap=50).save(f"prediction_22:28:23.171_raw.jpg.jpg")
print(predict)

None
